### Домашнее задание 
1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.
2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).
4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).
5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.
6. Могла ли модель переобучиться? Почему?

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

## Задание 1
*Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.


In [6]:
def calc_logloss(y, y_pred):
    if y_pred.dtype != 'float64':
        y_pred = np.array([y_pred], dtype=np.float64) # в случае если y_pred окажутся int
    y_pred[(y_pred>1-1e-10)] = 1-1e-10
    y_pred[(y_pred<1e-10)] = 1e-10   
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

проверка

In [7]:
y1 = np.array([1, 1])
y_pred1 = np.array([0, 1])

In [8]:
calc_logloss(y1, y_pred1)

11.51292546502023

## Задание 2
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

Функция потерь log loss монотонно убывающая функция зависящая от произведения <w,x>, поэтому её градиент ни когда не будет 
равен 0. И с ростом числа итераций модель стремясь продвинуться по градиентному спучку будет увеличивать вектор весов бесконечно. 
Поэтому формулировка задания на мой взгляд не совсем корректная. 
Чтобы определить минимальный log loss нужно ввести регуляризацию, котороя бы не позволила рости весам бесконечно, 
либо зафиксировать ошибку принудительно.
Я решил пойти по второму пути. Зафиксировал ошибку на у провне 0,2877 (средняя вероятность р+ = 0.75, np.log(0.75)) 


In [9]:
def eval_model_pred(X, y, eta, log=False):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    i = 0
    err = 1
    while err > 0.2877: # средняя вероятность P+ = 0.75
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        i +=1
        if i % 10 == 0 and log:
            print(i, W, err)
    return i, eta, err, W

обучим модель на разных скоростях обучения

In [10]:
eta = np.array([1e-3, 1e-2, 1e-1, 1, 2, 3])

In [11]:
res = []
for el_eta in eta:
    res.append(np.array(eval_model_pred(X_st, y, eta=el_eta), dtype='object'))
res = np.array(res)

рассмотрим res. Из 5 возможных вариантов при разной eta, мы сумели добиться необходимой нам ошибки. Хотя при eta > 1 можно наблюдать резкий рост значений весов. Поэтому в качестве оптимального выбираю 
eta = 1,
iteretions = 359

In [12]:
print('Iteretions, eta, err')
print(res[:,:3])

Iteretions, eta, err
[[366446 0.001 0.28769988694018844]
 [36645 0.01 0.28769976681419374]
 [3665 0.1 0.2876968899115469]
 [359 1.0 0.2876633146052598]
 [126 2.0 0.287453191695589]
 [125 3.0 0.28429827382131223]]


In [13]:
print('Веса W')
print(res[:,3])

Веса W
[array([-6.26210276, -1.06751079, -0.98117828,  5.65954682])
 array([-6.26219929, -1.06751589, -0.98121153,  5.65961737])
 array([-6.26326652, -1.06757252, -0.98157884,  5.66039777])
 array([-6.27422062, -1.06817129, -0.98531906,  5.66843845])
 array([-6.46935804, -1.2689934 , -0.72835177,  6.13799067])
 array([-10.12329296,  -2.08851649,  -0.99279209,  10.00304125])]


In [14]:
W = res[3,3]
W

array([-6.27422062, -1.06817129, -0.98531906,  5.66843845])

## Задание 3
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [15]:
def calc_pred_proba(X, W):
    z = np.dot(X, W)
    y_pred = sigmoid(z) 
    return y_pred

In [16]:
calc_pred_proba(X_st, W)

array([0.32987619, 0.24691228, 0.96718543, 0.00557079, 0.70949528,
       0.43777942, 0.98691902, 0.11573679, 0.35271092, 0.94662261])

## Задание 4
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [17]:
def calc_pred(X, W, b=0.5):
    y_predicted = np.zeros(X.shape[0])
    z = np.dot(X, W)
    y_pred = sigmoid(z) 
        # За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(y_pred.shape[0]):
        if (y_pred[i] > b): 
            y_predicted[i] = 1
        elif (y_pred[i] <= b):
            y_predicted[i] = 0
    return y_predicted

In [18]:
print(calc_pred(X_st, W, 0.5))
print(y)

[0. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]


## Задание 5
*Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

### accuracy

In [19]:
y_pred = calc_pred(X_st, W, 0.5)

In [20]:
def accuracy(y, y_pred):
    acu = 1 - np.mean(np.abs(y-y_pred))
    return acu

In [21]:
print(f'accuracy: {accuracy(y, y_pred)}')

accuracy: 0.9


### матрица ошибок

In [22]:
def matrix_err(y, y_pred):
    m = y.shape[0]
    mx = np.zeros((2,2))
    for index in range(m):
            if y[index]==1 and y_pred[index]==1:
                mx[0,0] +=1
            if y[index]==0 and y_pred[index]==0:
                mx[1,1] +=1
            if y[index]==0 and y_pred[index]==1:
                mx[0,1] +=1
            if y[index]==1 and y_pred[index]==0:
                mx[1,0] +=1                
    return mx

In [23]:
print(f'матрица ошибок:\n {matrix_err(y, y_pred)}')

матрица ошибок:
 [[4. 0.]
 [1. 5.]]


### Полнота

In [24]:
def recall(y, y_pred):
    mx = matrix_err(y, y_pred)
    return mx[0,0]/(mx[0,0]+mx[1,0])

In [25]:
print(f'Полнота (recall): {recall(y, y_pred)}')

Полнота (recall): 0.8


### Точность

In [26]:
def precision(y, y_pred):
    mx = matrix_err(y, y_pred)
    return mx[0,0]/(mx[0,0]+mx[0,1])

In [27]:
print(f'Точность (precision): {precision(y, y_pred)}')

Точность (precision): 1.0


### F_score

In [28]:
def f_score(y, y_pred, b=1):
    r = recall(y, y_pred)
    p = precision(y, y_pred)
    res = (1+b**2)*(r*p)/(b**2*𝑝+r)
    return res

In [29]:
print(f'f_score: {f_score(y, y_pred)}')

f_score: 0.888888888888889


## Задание 6
Могла ли модель переобучиться? Почему?

Да, могла.

Функция потерь log loss монотонно убывающая функция зависящая от произведения <w,x>, 
поэтому её градиент ни когда не будет равен 0. И с ростом числа итераций модель стремясь продвинуться по градиентному спуску будет увеличивать W бесконечно. 